In [1]:
####TODO:
# Train the discriminator against garbage, to validate. CHECK
# See what happens to the misclassified test data as training proceeds: 
#    The model should become more and more certain about its incorrect answer...
# it looks like the decision boundary is INSANELY sharp.  How can we "blur" that?  
#    Is this related to the "blurring" effect that I was thinking about before? (to soften the directional derivative?)

####NOTES:
# The output for the real data and uniform mnist-shaped data are indistinguishable from one another.
#    This suggests that the decision boundary is crazy sharp.  That is fine as long as the decision boundary
#    doesn't have too many "bends" in it.  However, the real data NEEDS lots of "bends" to be separated,
#    because it is weird in the original space.  How many "bends" is too many "bends"?
#    Can we use a metric of overfitting to determine this?

####DONE:
# Validation:::
#    Trained the discriminator against uniform mnist-shaped data on [0,1/255].
#        Results: recognizes training data with ~89% acc, random data with 100% acc
#    Trained the discriminator against uniform mnist-shaped data on [0,1].
#        Results: SUCCEEDS! ~90% acc training, 100% acc random data... THIS IS WEIRD BC THE OUTPUTS
#            LOOK NEARLY THE SAME!! !!!CHECK THE ENTROPY HISTOGRAMS!!!
#    Trained the discriminator against test data.
#        Results: failed to do better than 50/50
#  This suggests that the model architecture is fine, and the data is indistinguishable.
#  Furthermore, a simple inspection of the model output suggests that the vectors are not really different.
#  !!!Make graphs of the entropy to illustrate ^ this point.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
from mnist_models import *
from mnist_discriminator import *
from utils import *
import os

%matplotlib nbagg

#### SET THIS BEFORE RUNNING
MNIST_MODEL = "mnist_models/six_or_bust"
DISCRIMINATOR_MODEL = "mnist_discriminator/discriminator_01"

try:
    checkpoints = sorted(["{}/{}".format(ROOT.format(MNIST_MODEL),x[:-5]) for x in os.listdir(ROOT.format(MNIST_MODEL)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
except FileNotFoundError:
    checkpoints = []
print(len(checkpoints))


Extracting /home/scott/USA/overfitting_study/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/scott/USA/overfitting_study/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/scott/USA/overfitting_study/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/scott/USA/overfitting_study/data/MNIST_data/t10k-labels-idx1-ubyte.gz
4000


In [2]:
tf.reset_default_graph()
model = MnistConvolutionalModel()
model.saver.recover_last_checkpoints(checkpoints[-1:])
try: latest_checkpoint = model.saver.last_checkpoints[-1]
except: pass
discriminator = Discriminator()

In [3]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import DataSet
np.random.seed(0)
neworder = list(range(len(mnist.train.images)))
np.random.shuffle(neworder)
images = np.array([mnist.train.images[i] for i in neworder])
labels = np.array([mnist.train.labels[i] for i in neworder])
traintrainimages = images[:10000]
traintrainlabels = labels[:10000]
trainvalimages = images[-5000:]
trainvallabels = labels[-5000:]
mnisttraintrain = DataSet(traintrainimages*255., traintrainlabels, reshape=False) #########THIS IS DIVIDING EVERYTHING BY 255!!!
mnisttrainval = DataSet(trainvalimages*255., trainvallabels, reshape=False)
uniform_mnist_shaped_data = DataSet(np.random.randint(0,256,mnist.test.images.shape), np.random.randint(0,2,mnist.test.labels.shape), reshape=False)

In [4]:
discriminator.train(model, latest_checkpoint, mnisttraintrain, uniform_mnist_shaped_data, DISCRIMINATOR_MODEL)

KeyboardInterrupt: 

In [2]:
test_batch_labels = np.array([(1,0)]*50)
train_batch_labels = np.array([(0,1)]*50)

HEADER = ["batch acc.", "train acc.", "test acc.", "save path"]
printtable(16, *HEADER)

MAX_N = 1000000
TRAIN_RATE_BEGIN = 0.001
TRAIN_RATE_END = 0.00001
TRAIN_RATE_EXPONENT = (TRAIN_RATE_BEGIN*1./TRAIN_RATE_END)**(1./MAX_N)
with tf.Session() as sess:
    discriminator.initialize(sess)
    discriminator.set_train_rate(TRAIN_RATE_BEGIN)

    model.saver.restore(sess, latest_checkpoint)
    for i in range(1000000):
        table = []
        # do training or not training data every other batch
        if i%2:
            # train the discriminator on some training data
            batch = mnisttraintrain.next_batch(50)
            blabels = train_batch_labels
        else:
            # train the discriminator on some non-training data
            batch = mnist.test.next_batch(50)
            #batch = uniform_mnist_shaped_data.next_batch(50)
            blabels = test_batch_labels
        probas = model.proba.eval(feed_dict={
            model.x:batch[0],
            model.y_:batch[1],
            model.keep_prob:1.0})
        if not i%501:
            print(batch[0][0])
            print(model.proba.eval(feed_dict={
                model.x:batch[0][:1],
                model.y_:batch[1][:1],
                model.keep_prob:1.0
            }))
            print(
                  discriminator.logits.eval(feed_dict={
                    discriminator.x: probas[:1],
                    discriminator.y_: blabels[:1],
                    discriminator.keep_prob:1.0
                  })
                  
            )
            batch_accuracy = discriminator.accuracy.eval(feed_dict={
                discriminator.x: probas, 
                discriminator.y_: blabels,
                discriminator.keep_prob:1.0
            })
        discriminator.train_step.run(feed_dict={
            discriminator.x: probas, 
            discriminator.y_: blabels,
            discriminator.keep_prob: 0.5
        })
        if not i%10000:
            discriminator.set_train_rate(discriminator.train_rate*TRAIN_RATE_EXPONENT)
        if not i%501:
            pr = model.proba.eval(feed_dict={
                model.x: mnisttraintrain.images,
                model.y_: mnisttraintrain.labels,
                model.keep_prob:1.0
            })
            train_accuracy = discriminator.accuracy.eval(feed_dict={
                discriminator.x: pr,
                discriminator.y_: np.array([(0,1)]*10000),
                discriminator.keep_prob:1.0
            })
        
            pr = model.proba.eval(feed_dict={
                model.x: mnist.test.images,
                model.y_: mnist.test.labels,
                #model.x: uniform_mnist_shaped_data.images,
                #model.y_: uniform_mnist_shaped_data.labels,
                model.keep_prob:1.0
            })

            test_accuracy = discriminator.accuracy.eval(feed_dict={
                discriminator.x: pr,
                discriminator.y_: np.array([(1,0)]*10000),
                discriminator.keep_prob:1.0
            })
            
            save_path = discriminator.saver.save(sess, "./discriminator/discriminator_test2/model.ckpt", global_step=i)
            print(("{}"*len(HEADER)).format(*printtable(16, batch_accuracy, train_accuracy, test_accuracy, save_path)))
                  
                
                

NameError: name 'np' is not defined

In [8]:
with tf.Session() as sess:
    discriminator.saver.restore(sess, "./discriminator/discriminatorB/model.ckpt-9500")
    model.saver.restore(sess, latest_checkpoint)
    
    pr = model.proba.eval(feed_dict={
        model.x: mnisttraintrain.images,
        model.y_: mnisttraintrain.labels,
        model.keep_prob:1.0
    })
    ans = discriminator.accuracy.eval(feed_dict={
        discriminator.x: pr,
        discriminator.y_: np.array([(0,1)]*10000),
        discriminator.keep_prob:1.0
    })
    print(ans)
    pr = model.proba.eval(feed_dict={
        model.x: mnist.test.images,
        model.y_: mnist.test.labels,
        model.keep_prob:1.0
    })
    ans = discriminator.accuracy.eval(feed_dict={
        discriminator.x: pr,
        discriminator.y_: np.array([(1,0)]*10000),
        discriminator.keep_prob:1.0
    })
    print(ans)
    
    
    print(accuracy.eval(feed_dict={model.x: trainvalimages, model.y_: trainvallabels, model.keep_prob:1.0}))
    print(discriminator.b2.eval())
    print(model.b_fc2.eval())
    trainers = model.proba.eval(feed_dict={
        model.x: mnisttraintrain.images[5000:10000],
        model.y_: mnisttraintrain.labels[5000:10000],
        model.keep_prob:1.0
    })    
    print(trainers[0])
    print(labels[5000])
    validaters = model.proba.eval(feed_dict={
        model.x: mnist.validation.images,
        model.y_: mnist.validation.labels,
        model.keep_prob:1.0
    })
    print(validaters[0])
    print(mnist.validation.labels[0])
    trainacc = discriminator.accuracy.eval(feed_dict={
        discriminator.x: trainers,
        discriminator.y_: np.array([(0,1)]*5000),
        discriminator.keep_prob: 1.0
    })
    valacc = discriminator.accuracy.eval(feed_dict={
        discriminator.x: validaters,
        discriminator.y_: np.array([(1,0)]*5000),
        discriminator.keep_prob: 1.0
    })
    print(trainacc, valacc)
    print(discriminator.proba.eval(feed_dict={discriminator.x: trainers[0:2], discriminator.y_: np.array([(0,1)]*2), discriminator.keep_prob:1.0}))
    print(discriminator.proba.eval(feed_dict={discriminator.x: validaters[0:2], discriminator.y_: np.array([(1,0)]*2), discriminator.keep_prob:1.0}))
    
    pr = model.proba.eval(feed_dict={
        model.x: mnist.train.images[0:2],
        model.y_: mnist.train.labels[0:2],
        model.keep_prob:1.0
    })
    ans = discriminator.proba.eval(feed_dict={
        discriminator.x: pr,
        discriminator.y_: np.array([(0,1)]*2),
        discriminator.keep_prob:1.0
    })
    print(pr)
    print(ans)
    

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ./discriminator/discriminatorB/model.ckpt-9500
	 [[Node: save_1/RestoreV2_1 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2_1/tensor_names, save_1/RestoreV2_1/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dis

In [ ]:

with tf.Session() as sess:
    model.saver.restore(sess, latest_checkpoint)
    train_entropies = list(map(entropy, model.proba.eval(feed_dict={
        model.x: mnisttraintrain.images,
        model.y_: mnisttraintrain.labels,
        model.keep_prob: 1.0
    })))
    test_entropies = list(map(entropy, model.proba.eval(feed_dict={
        model.x: mnist.test.images,
        model.y_: mnist.test.labels,
        model.keep_prob: 1.0
    })))
    plt.hist(np.log2(np.array(test_entropies)*-1), bins=10, color='b', alpha=0.5)
    plt.hist(np.log2(np.array(train_entropies)*-1), bins=10, color='r', alpha=0.5)
    plt.show()
